# Link prediction in citation graphs

In [2]:
import random
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import nltk

nltk.download('stopwords')
stpwds = set(nltk.corpus.stopwords.words("english"))


import pandas as pd

from numpy import linalg as LA

import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pylab
import sklearn
import scipy
from scipy import stats


import os
import PIL
from PIL import Image
import pickle
import math
from tempfile import TemporaryFile
import re
%matplotlib inline

import random
random.seed(0)
import nltk
import string
import re
nltk.download('stopwords')
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MARIVAL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MARIVAL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Loading and visualizing the data

In [6]:
trainDF = pd.read_csv("training_set.txt",header=None,sep=" ")
trainDF.columns=['source','target','label']
testDF = pd.read_csv("testing_set.txt",header=None,sep=" ")
testDF.columns=['source','target']
node_infoDF = pd.read_csv("node_information.csv",header=None)
node_infoDF.columns=['ID','year','title','authors','journal','abstract']

print("shapes : train={}, test={}, nodes={}".format(trainDF.shape, testDF.shape, node_infoDF.shape))
print (trainDF.head(2))
print (testDF.head(2))
node_infoDF.head(10)

shapes : train=(615512, 3), test=(32648, 2), nodes=(27770, 6)
    source   target  label
0  9510123  9502114      1
1  9707075  9604178      1
    source   target
0  9807076  9807139
1   109162     1182


,ID,year,title,authors,journal,abstract
0,1001,2000,compactification geometry and duality,Paul S. Aspinwall,NaN,these are notes based on lectures given at tas...
1,1002,2000,domain walls and massive gauged supergravity p...,"M. Cvetic, H. Lu, C.N. Pope",Class.Quant.Grav.,we point out that massive gauged supergravity ...
2,1003,2000,comment on metric fluctuations in brane worlds,"Y.S. Myung, Gungwon Kang",NaN,recently ivanov and volovich hep-th 9912242 cl...
3,1004,2000,moving mirrors and thermodynamic paradoxes,Adam D. Helfer,Phys.Rev.,quantum fields responding to moving mirrors ha...
4,1005,2000,bundles of chiral blocks and boundary conditio...,"J. Fuchs, C. Schweigert",NaN,proceedings of lie iii clausthal july 1999 var...
5,1006,2000,questions in quantum physics,Rudolf Haag,NaN,an assessment of the present status of the the...
6,1007,2000,topological defects in 3-d euclidean gravity,"Sheng Li, Yong Zhang, Zhongyuan Zhu",NaN,by making use of the complete decomposition of...
7,1008,2000,n 0 supersymmetry and the non-relativistic mon...,Donald Spector,Phys.Lett.,we study some of the algebraic properties of t...
8,1009,2000,gluon pair production from space-time dependen...,"Gouranga C. Nayak, Walter Greiner",NaN,we compute the probabilty for the processes a ...
9,1010,2000,instantons euclidean supersymmetry and wick ro...,"A.V. Belitsky, S. V, oren, P. van Nieuwenhuizen",Phys.Lett.,we discuss the reality properties of the fermi...


In [3]:
node_infoDF.isnull().sum()

ID             0
year           0
title          0
authors     4033
journal     7472
abstract       0
dtype: int64

## Designing features

Since very few features were usable in the raw data, we had to design new features from this raw data. We'll start by listing the features we designed, then we will try different classification algorithms with those features and tune the parameters of the best one.

## First set of features - Nodeinfo, graph structure

### Abstract words tfidf

Sylvain: To compare two abstracts and measure how "close" they are to each other, we will use tfidf on the words in the abstract. We first make a list of the words in all the abstracts in order to compute the document frequency. This is the role of the dictionary dict0.

In [3]:
def unique(a):
    k=0
    while k < len(a):
        if a[k] in a[k+1:]:
            a.pop(k)
        else:
            k=k+1
            
    return a

docfr will be used for the document frequency in the tfidf

In [35]:
for i in range(len(dict0)):
    b=0
    for j in range(len(nodeinf)):
        dict1=[]
        dict1=dict1+re.split('\s+', nodeinf[5][i])
        if dict0[0][i] in dict1:
            b=b+1
    dict0["docfr"][i]=b/len(nodeinf)

    print(i)

Sylvain:We will now remove the words with a small tfidf score (<1.7) since they are not very relevant. Then thanks to this dictionary we will be able to compute how many words with high tfidf score 2 documents have in common.

In [ ]:
def applyfunc5(x):
    x1=[]
    x2=[]
    for g2 in x:
        if x[g2]>1.7:
            x1=x1+[g2]
            x2=x2+[x[g2]]
    dico = dict(zip(x1,x2))
    return(dico)

In [ ]:
nodeinf["tfidf1"]=0
nodeinf["tfidf1"]=nodeinf["tfidf"].apply(applyfunc5)

In [ ]:
import nltk
import string
import re
nltk.download('stopwords')
stpwds = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.stem.PorterStemmer()
punct = string.punctuation.replace("-", "")

def tokenize(content, remove_stopwords=True, stemming=True):
    # remove formatting
    content =  re.sub("\s+", " ", content)
    # convert to lower case
    content = content.lower()
    # remove punctuation (preserving intra-word dashes)
    content = "".join(letter for letter in content if letter not in punct)
    # remove dashes attached to words but that are not intra-word
    content = re.sub("[^[:alnum:]['-]", " ", content)
    content = re.sub("[^[:alnum:][-']", " ", content)
    # remove extra white space
    content = re.sub(" +"," ", content)
    # remove leading and trailing white space
    content = content.strip()
    # tokenize
    tokens = content.split(" ")
    # remove stopwords
    if remove_stopwords==True:
        tokens = [token for token in tokens if token not in stpwds]
    if stemming==True:
        tokens = [stemmer.stem(token) for token in tokens]
    return tokens

### Number of common authors

Let us define the variable containing the number of common authors

In [10]:
training_set2 = pd.DataFrame()
training_set2["num_com_aut"]=0

In [11]:
def numcomau(x):
    a1=x["source"]
    str1=nodeinf[nodeinf[0]==a1]
    str1=str1.reset_index(drop=True)
    str1=str1.loc[0,3]
    if pd.isnull(str1)==False:
        b1=re.split(',',str1)
        cst=0
        a2=x["target"]
        str2=nodeinf[nodeinf[0]==a2]
        str2=str2.reset_index(drop=True)
        str2=str2.loc[0,3]   
        if pd.isnull(str2)==False:
            b2=re.split(',', str2)
            for j in range(len(b1)):
                for k in range(len(b2)):
                    if tokenize(b1[j])==tokenize(b2[k]):
                        cst=cst+1
            return(cst)
        else:
             return(0)
    else:
        return(0)
    

In [12]:
training_set2["num_com_aut"]=training_set2.apply(numcomau,axis=1)

### Time difference (difference between the dates of publication)

Let us define the variable containing the time difference

In [17]:
training_set2["tempdiff"]=0

In [18]:
def numcomau1(x):
    a1=x["source"]
    str1=nodeinf[nodeinf[0]==a1]
    str1=str1.reset_index(drop=True)
    str1=int(str1.loc[0,1])
    if pd.isnull(str1)==False:
        a2=x["target"]
        str2=nodeinf[nodeinf[0]==a2]
        str2=str2.reset_index(drop=True)
        str2=int(str2.loc[0,1])   
        if pd.isnull(str2)==False:
            return(str1-str2)
        else:
            return(1)
    else:
        return(1)


In [20]:
training_set2["tempdiff"]=training_set2.apply(numcomau1,axis=1)

### Number of common words in the title

Let us define a varibale containing the number of common words in two documents' titles

In [21]:
training_set2["numcomwordintitle"]=0

In [22]:
def numcomau2(x):
    a1=x["source"]
    str1=nodeinf[nodeinf[0]==a1]
    str1=str1.reset_index(drop=True)
    str1=str1.loc[0,2]
    if pd.isnull(str1)==False:
        b1=re.split(' ',str1)
        cst=0
        a2=x["target"]
        str2=nodeinf[nodeinf[0]==a2]
        str2=str2.reset_index(drop=True)
        str2=str2.loc[0,2]   
        if pd.isnull(str2)==False:
            b2=re.split(' ',str2)
            for j in range(len(b1)):
                for k in range(len(b2)):
                    if b1[j]==b2[k]:
                        cst=cst+1
            return(cst)
        else:
            return(0)
    else:
         return(0)

In [23]:
training_set2["numcomwordintitle"]=training_set2.apply(numcomau2,axis=1)

### Were the articles published in the same journal

Let us define a categorical variable equal to 1 if two artcles were published in the same article, 0 otherwise.

In [24]:
training_set2["samejournal"]=0

In [25]:
def numcomau3(x):
    a1=x["source"]
    str1=nodeinf[nodeinf[0]==a1]
    str1=str1.reset_index(drop=True)
    str1=str1.loc[0,4]
    if pd.isnull(str1)==False:
        
        cst=0
        a2=x["target"]
        str2=nodeinf[nodeinf[0]==a2]
        str2=str2.reset_index(drop=True)
        str2=str2.loc[0,4]   
        if pd.isnull(str2)==False:

            if str1==str2:
                cst=1
            return(cst)
        else:
            return(0)
    else:
        return(0)

In [26]:
training_set2["samejournal"]=training_set2.apply(numcomau3,axis=1)

### Number of common words in the abstract with a high tfidf score (>1.7)

In [27]:
training_set2["numofightfidfwordsincom"]=0

In [28]:
def numcomau4(x):
    a1=x["source"]
    str1=nodeinf[nodeinf[0]==a1]
    str1=str1.reset_index(drop=True)
    str1=str1.loc[0,"tfidf1"]
    if pd.isnull(str1)==False:
        
        cst=0
        a2=x["target"]
        str2=nodeinf[nodeinf[0]==a2]
        str2=str2.reset_index(drop=True)
        str2=str2.loc[0,"tfidf1"]
        if pd.isnull(str2)==False:

            for j in str1:
                for k in str2:
                    if j==k:
                        cst=cst+1
            return(cst)
        else:
             return(0)
    else:
        return(0)

In [29]:
training_set2["numofightfidfwordsincom"]=training_set2.apply(numcomau4,axis=1)

### Number of times the author of the source paper cited a paper published in the same journal as the target paper

In [30]:
training_set2["nbdefoisautcitejournarttarget"]=0

In [31]:
for i in range(len(training_set2)):
    a1=training_set2["source"][i]
    str1=nodeinf[nodeinf[0]==a1]
    str1=str1.reset_index(drop=True)
    str1=str1.loc[0,3]
    if pd.isnull(str1)==False:
        str1=removepar(str1)
        b1=re.split(',',str1)
        a2=training_set2["target"][i]
        str2=nodeinf[nodeinf[0]==a2]
        str2=str2.reset_index(drop=True)
        str2=str2.loc[0,4]   
        if pd.isnull(str2)==False:
            cst=0
            j1=0
            for k in b1:
                if len(k)>3:
                    if k in list(dictaut[0]):
                        str3=dictaut[dictaut[0]==k].reset_index(drop=True).loc[0,"papers"]
                        if str2 in str3:
                            cst=cst+str3[str2]
                    else:

                        for j in range(len(dictaut)):
                            if k in dictaut[0][j]:
                                str3=dictaut .loc[j,"papers"]
                                if str2 in str3:
                                    cst=cst+str3[str2]
                                j1=1+j1
                    
            cst=cst/(len(b1)+j1)
            training_set2["nbdefoisautcitejournarttarget"][i]=cst
        else:
             training_set2["nbdefoisautcitejournarttarget"][i]=0
    else:
        training_set2["nbdefoisautcitejournarttarget"][i]=0
    if i%100==0:
        print(i/len(training_set2)*100)

### Depth-first	search

Sylvain :Shortest distance between two nodes. The code is quite long and boring so we didn't put it here. You can however find it in the annexe (ALTEGRAG.ipynb). We count how many edges we have to go through in order to go from a node to another. In our case some nodes the distance is only 1 as one paper can cite another. In order for the algorithm to predict we should not take into account direct connection between nodes. Indeed we dont have this information on the testing set. Therefore we have adapted the algorithm so that it doesn't take into account 2 adjacent nodes. We computed this algorithm by multiplying the adjacency matrix. As this matrix is very large, we used dictionaries to compute the multiplication.

## Feature X - Pagerank and HITS

We will attempt to pagerank the papers (nodes of the citation graph) to create a feature that we will help us predict missing citation links between papers. We will also compute the authors graph (there is a link from A to B if A has cited B). The authors pagerank and their hits score (hubness and authority) will also be used as new features.

### Paper pagerank

We first create the paper/citation graph with the training data trainDF. We used Networkx to create the graph: http://networkx.github.io/documentation/latest/_downloads/networkx_reference.pdf

In [5]:
import networkx as nx

In [6]:
GraphData = nx.DiGraph()
for i in range(int(trainDF.shape[0])):
    source_ID = trainDF.source[i]
    target_ID = trainDF.target[i]
    label = trainDF.label[i]
    if (label==1):
        GraphData.add_edge(source_ID,target_ID) 

In [7]:
GraphData

In order to avoid too much computing, we saved useful variables in a pickle file:

In [9]:
import pickle
f = open('pagerankVariables.p', 'wb')
pickle.dump(GraphData, f)
f.close()

Let us now compute the pagerank.

In [10]:
pr = nx.pagerank(GraphData)

In [13]:
#converts dictionary to dataframe 
prDF = pd.DataFrame(list(pr.items()), columns = ['source','pagerank'])
prDF.head()

,source,pagerank
0,9306112,0.000011
1,9306114,0.000011
2,9306115,0.000023
3,9306116,0.000018
4,9306117,0.000065


In [14]:
f = open('pagerankVariables.p', 'wb')
pickle.dump(prDF, f)
f.close()

### Author ranking

Lets create the author graph.

In [15]:
def cited_auths(id1,id2):
    source_auth = node_infoDF.ix[node_infoDF.ID==id1,"authors"]
    target_auth = node_infoDF.ix[node_infoDF.ID==id2,"authors"]
    if source_auth.isnull().values or target_auth.isnull().values: #a simplifier
        return 0
    else:
        return (set(source_auth.values[0].split(",")),set(target_auth.values[0].split(",")))
cit_test = cited_auths(1001,1002)

In [16]:
cit_test

({'Paul S. Aspinwall'}, {' C.N. Pope', ' H. Lu', 'M. Cvetic'})

This function takes the IDs of two papers and returns a couple where the first element is the set of authors who wrote the source article and the second element the set of authors who wrote the target article. To create the authors graph we will create a link between every pair (source_author,target_author).

In [17]:
GraphAuths = nx.DiGraph()
for i in range(int(trainDF.shape[0])):
    source_ID = trainDF.source[i]
    target_ID = trainDF.target[i]
    label = trainDF.label[i]
    if (label == 1):
        cited_list = cited_auths(source_ID, target_ID)
        if (cited_list == 0):
            pass
        else:
            for source_author in list(cited_list[0]):
                for target_author in list(cited_list[1]):
                    GraphAuths.add_edge(source_author,target_author)  

In [18]:
f = open('pagerankVariables.p', 'wb')
pickle.dump(GraphAuths, f)
f.close()

Computing the pagerank :

In [19]:
auth_pr = nx.pagerank(GraphAuths)

In [20]:
#converts dictionary to dataframe 
auth_prDF = pd.DataFrame(list(auth_pr.items()), columns = ['author','author_pr'])
auth_prDF.head()

,author,author_pr
0,Brett J. Taylor,0.000011
1,A. Sugamoto,0.000015
2,Mariano J. Salvay,0.000010
3,M.N. Stoilov,0.000010
4,T. Kojima,0.000070


In [21]:
f = open('pagerankVariables.p', 'wb')
pickle.dump(auth_prDF, f)
f.close()

Computing the HITS scores :

In [22]:
auth_hits = nx.hits(GraphAuths)

nx.hits(GraphAuths) returns a couple containing two dictionnaries, the first one (auth_hits[0]) with the hubnesses of each node, the second one (auth_hits[1]) with the authorities of the nodes.

In [23]:
#converts dictionary to dataframe 
auth_hubsDF = pd.DataFrame(list(auth_hits[0].items()), columns = ['author','author_hubs'])
auth_hubsDF.head()

,author,author_hubs
0,Brett J. Taylor,9.540999e-05
1,A. Sugamoto,6.462332e-05
2,Mariano J. Salvay,8.582825e-07
3,J. Maldacena,8.747089e-04
4,T. Kojima,8.430559e-06


In [24]:
f = open('pagerankVariables.p', 'wb')
pickle.dump(auth_hubsDF, f)
f.close()

In [25]:
#converts dictionary to dataframe 
auth_authoritiesDF = pd.DataFrame(list(auth_hits[1].items()), columns = ['author','author_authorities'])
auth_authoritiesDF.head(10)

,author,author_authorities
0,Brett J. Taylor,0.000002
1,A. Sugamoto,0.000028
2,Mariano J. Salvay,0.000000
3,J. Maldacena,0.001670
4,T. Kojima,0.000007
5,K.Selivanov,0.000003
6,P. Brax,0.000031
7,M.Tarlini,0.000019
8,Yasuyuki Sekiguchi,0.000012
9,Hiroshi Ishikawa (Tohoku Univ.),0.000017


In [26]:
f = open('pagerankVariables.p', 'wb')
pickle.dump(auth_authoritiesDF, f)
f.close()